## 1. Create endpoint config
In this section we will describe how we can create a model endpoint.
If you already have endpoint , please go to last cell for invocation, give the correct end point name and start inferences.

For generating sagemaker end point we will need to generate endpoint config. So generating endpoint config first

In [1]:
%store -r # you can read model from another notebook via this method
print(v_model_name)

ds-mlops-linear-learner


In [3]:
import sys
import math
import boto3
import sagemaker
from time import gmtime, strftime
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
runtime_client = boto3.client("runtime.sagemaker", region_name=region)
client = boto3.client("sagemaker", region_name=region)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [2]:
# Instance config for endpoint
endpoint_config_name = 'ds-mlops-XGBoost-endpoint-configV1'
v_model_name = "wipcoe-pricing-ml-training-pipeline-xg-3196968130"
Instance_Type = "ml.m5.large"
VolumeSizeIn_GB = 20
Instance_Count = 1
endpoint_name = 'ds-mlops-XGBoost-endpoint' # name of model endpoint
datacaploc='s3://ds-mlops-s3/data/capture/'

**Note :** We will use able endpoint config name to cretae model endpoint

## 2. Delete endpoint config

Incase if you want to delete endpoint config and recreate it we can use below code snippet

In [ ]:
# response = client.delete_endpoint_config(
#     EndpointConfigName=endpoint_config_name
# )

In [3]:
# Creating endpoint config
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": Instance_Type,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": v_model_name,
            "VariantName": "AllTraffic",
        }
    ],
    DataCaptureConfig={
        'EnableCapture': True,
        'InitialSamplingPercentage': 100,
        'DestinationS3Uri': datacaploc,
        'CaptureOptions': [
            {
                'CaptureMode': 'Input'
            },
             {
                'CaptureMode': 'Output'
            },
        ],
        'CaptureContentTypeHeader': {
            'CsvContentTypes': [
                'text/csv',
            ],
            'JsonContentTypes': [
                'application/json',
            ]
        }        
    }
)

print(f"Endpoint Config Arn: {create_endpoint_config_response['EndpointConfigArn']}")

ds-mlops-XGBoost-endpoint-configV1
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:525102048888:endpoint-config/ds-mlops-xgboost-endpoint-configv1


## 3. Delete endpoint
Incase if you want to delete endpoint  and recreate it we can use below code snippet

In [ ]:
client.delete_endpoint(EndpointName=endpoint_name)

## 4.Create Endpoint

In this notebook we create endpoint using endpoint which we have created

In [4]:
# Importing required library
import sys
import math
import boto3
import sagemaker
import time    

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
runtime_client = boto3.client("runtime.sagemaker", region_name=region)
client = boto3.client("sagemaker", region_name=region)


In [5]:
# Creating model endpoint
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
while status == "Creating":
    print(f"Status: {status}")
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]

print(f"Arn: {resp['EndpointArn']}")
print(f"Status: {status}")

ds-mlops-XGBoost-endpoint
arn:aws:sagemaker:us-east-1:525102048888:endpoint/ds-mlops-xgboost-endpoint
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Arn: arn:aws:sagemaker:us-east-1:525102048888:endpoint/ds-mlops-xgboost-endpoint
Status: InService


## 5.Inference

In this section we will validate scoring logics. In below code snippet we are scoring with 14 predictor values and finding the result

In [7]:
payload={'year': 2018, 'manufacturer': 5, 'model': '147', 'condition': '2', 'cylinders': '4', 'fuel': '2', 'odometer': '2000', 'title_status': 0, 'transmission': '2', 'drive': '3', 'type': '8', 'paint_color': '6'}
scoring_vector = [payload['year'],payload['manufacturer'],payload['model'],payload['condition'],payload['cylinders'],payload['fuel'],payload['odometer'],payload['title_status'],payload['transmission'],payload['drive'],payload['type'],payload['paint_color']]
xgbody = ','.join([str(item) for item in scoring_vector])
client = boto3.client('sagemaker-runtime')
response_model_xgboost = client.invoke_endpoint(EndpointName=endpoint_name,
                               ContentType='text/csv',
                               Body=xgbody)
result_xgboost = float(response_model_xgboost["Body"].read())
print(result_xgboost)

8060


In [100]:
payload={'year': 2018, 'manufacturer': 5, 'model': '147', 'condition': '2', 'cylinders': '4', 'fuel': '2', 'odometer': '2000', 'title_status': 0, 'transmission': '2', 'drive': '3', 'type': '8', 'paint_color': '6'}
scoring_vector = [0.591345486,5.424464379,0.950671615,0.581766684,1.023412717,8.364105669,0.080246146,10,0,0]
xgbody = ','.join([str(item) for item in scoring_vector])
print(xgbody)
#payload=0.591345486,0.424464379,0.250671615,0.581766684,1.023412717,0.364105669,0.080246146,1,0,0
client = boto3.client('sagemaker-runtime')
response_model_xgboost = client.invoke_endpoint(EndpointName="MLOPsDemo-prod",
                               ContentType='text/csv',
                               Body=xgbody)
result_xgboost = float(response_model_xgboost["Body"].read())
print(result_xgboost)

0.591345486,5.424464379,0.950671615,0.581766684,1.023412717,8.364105669,0.080246146,10,0,0
11.862833023071289


**Note:** Similarly you can change the test vector and can see how endpoint is working